# MNIST Classifier

In this notebook you will create both, an mnist tabular dataset and a classifier.

## 1.- import the Operating System (os) module in python and any other library you need

In [1]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import time 

## 2.- As you can see each class has its own folder (Do it only for train). 

    - Iterate folder by folder ( os.listdir() )
    - Inside each folder: 
        1.- Read the image
        2.- Reshape it into a flat array (784,)
        3.- Save the data into a pandas dataframe apending the column name as the class
    - Save the data into a CSV

    Note: if it takes to long try doing only 100 images per folder and the teacher for the CSV.

In [2]:
base_path = 'trainingSet/trainingSet'

'''
path = base_path + '/' + folder + '/' + img_name
'''
df_arr=[]
for folder in os.listdir(base_path):
    # print(folder)
    im_folder = os.path.join(base_path, folder)
    for im_name in os.listdir(im_folder):
        im_path = os.path.join(im_folder, im_name)
        path=Image.open(im_path)

        arr=np.array(path, dtype=float).flatten()
        arr=np.append(arr,int(folder))
        df_arr.append(arr)


df=pd.DataFrame(df_arr)

    

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,3.0,0.0,0.0,3.0,7.0,3.0,0.0,3.0,0.0,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
x,y =df.drop(784, axis=1), df[784]

## 3.- Load the CSV

## 4.- Create a dictionary of models (No preprocessing needed, it has already been done).
    
    Include both, tree models and mult models.

In [8]:
from sklearn.model_selection import  train_test_split ,cross_val_predict, StratifiedKFold
from sklearn.metrics import  accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn.pipeline import Pipeline , make_pipeline

In [12]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from sklearn.svm           import SVC
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier
from sklearn.neighbors     import KNeighborsClassifier

In [13]:
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(random_state=0),
  "Extra Trees":   ExtraTreesClassifier(random_state=0),
  "Random Forest": RandomForestClassifier(random_state=0),
  "AdaBoost":      AdaBoostClassifier(random_state=0),
  #'KNN'     :       KNeighborsClassifier(n_neighbors=9),
  "Skl GBM":       GradientBoostingClassifier(random_state=0),
  "Skl HistGBM":   HistGradientBoostingClassifier(random_state=0),
  "XGBoost":       XGBClassifier(),
  "LightGBM":      LGBMClassifier(random_state=0),
  "CatBoost":      CatBoostClassifier(n_estimators=100, allow_writing_files=False)
}

tree_classifiers = {name: make_pipeline(model) for name, model in tree_classifiers.items()}

In [14]:
results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

x_train, x_test, y_train, y_test = train_test_split(x, y,random_state=0,test_size=0.2,stratify=y)

for model_name, model in tree_classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    model.fit(x_train, y_train)
    # GET PREDICTIONS USING x_val
    pred = model.predict(x_test)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": accuracy_score(y_test, pred)*100,
                              "Bal Acc.": balanced_accuracy_score(y_test, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)



results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

C:\Users\KINGSLEY\AppData\Local\Temp\ipykernel_15236\335486925.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"Model":    model_name,


## 5.- Using either cross validation or stratification find out which is the best model
    - Base your code on the previous two days examples

In [ ]:




results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})



skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)



for model_name, model in tree_classifiers.items():

    start_time = time.time()
    pred = cross_val_predict(model, x, y, cv=skf)
    total_time = time.time() - start_time
    results = results.append({"Model":    model_name,
                              "Accuracy": accuracy_score(y, pred)*100,
                              "Bal Acc.": balanced_accuracy_score(y, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

## Optional: Can you rotate an image?